In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import json
import tensorflow as tf
import numpy as np

from mrcnn import visualize
import mrcnn.model as modellib
from mrcnn.model import log
import perception
import warnings
warnings.filterwarnings("ignore")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)
tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print(tf.config.list_physical_devices())

1 Physical GPUs, 1 Logical GPUs
Num GPUs Available:  1
Num CPUs Available:  1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-02-20 18:43:35.697542: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 18:43:35.723392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 18:43:35.723572: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 18:43:35.724273: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
config = perception.CocoConfig()
class_names = ['BG', 'person', 'rider', 'car', 'bus', 'truck', 'bike', 'motor', 'traffic light', 'traffic sign', 'train']

In [4]:
image_dir = "/run/media/sam/Dataset/bdd100k-models/data/bdd100k/images/100k/test/"
coco_dir = "/home/sam/Desktop/cocoData/"
output_image_path = "/home/sam/Desktop/output/"
weights_path = "/home/sam/Desktop/SamHons_ObjectDetection_Repo/object_detection/training_log_rgb/coco20190811T1837/mask_rcnn_coco_0005.h5"
MODEL_DIR = "/home/sam/Desktop/SamHons_ObjectDetection_Repo/object_detection/training_log_rgb"

In [5]:
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights(weights_path, by_name=True)

Instructions for updating:
Use fn_output_signature instead


ValueError: Layer #389 (named "mrcnn_bbox_fc"), weight <tf.Variable 'mrcnn_bbox_fc/kernel:0' shape=(1024, 44) dtype=float32, numpy=
array([[-0.07423402, -0.01301147,  0.03234645, ..., -0.04311334,
         0.03385313,  0.02864521],
       [ 0.06854087, -0.03344586,  0.06797224, ...,  0.05952951,
        -0.00699303,  0.00632061],
       [-0.05997106,  0.04689942,  0.04211205, ..., -0.04971891,
         0.05169499,  0.04476684],
       ...,
       [-0.06145637, -0.06109542, -0.07162572, ...,  0.0202061 ,
        -0.03206471, -0.0645671 ],
       [-0.01481646,  0.06420629,  0.02765223, ..., -0.0666992 ,
        -0.06287075, -0.00632829],
       [ 0.02516412,  0.03527036,  0.0312301 , ..., -0.05117411,
        -0.0243571 ,  0.06873274]], dtype=float32)> has shape (1024, 44), but the saved weight has shape (1024, 324).

In [ ]:
def get_image_list(image_dir):
    files= os.listdir(image_dir) 
    s = []
    for file in files: 
        str_name = file[:21]
        s.append(str_name) 
    return s

image_list = get_image_list(image_dir)

In [ ]:
def get_individual_class_details(class_ids, scores, rois):
    
    result_array = dict()
    for index in range(len(class_ids)):

        predicted_class = class_names[class_ids[index]]
        predicted_class_score = scores[index].tolist()
        predicted_class_boxes = rois[index].tolist()

        result_array[predicted_class] = ({"Confidence":[predicted_class_score]}, 
                                                 {"Bounding box":[predicted_class_boxes]})

    #print(result_array)
    return result_array

In [ ]:
test_annos = dict()
JSON_PATH_OUTPUT = os.path.join(output_image_path, "results.json")
for i in range(len(image_list)):
    image_path = os.path.join(image_dir,image_list[i]) 
    print(image_path)
    image = cv2.imread(image_path)

    height, width = image.shape[:2]

    results = model.detect([image], verbose=1)


    r = results[0]
    image_ir = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], 
                                title="Predictions")
    plt.savefig(os.path.join(output_image_path,image_list[i]), bbox_inches='tight')
     
    class_details = get_individual_class_details(r['class_ids'], r['scores'], r['rois'])
    test_annos[i] = ({image_list[i] : class_details})
    
    #test_annos[image_list[i]] = (get_individual_class_details(r['class_ids'], r['scores'], r['rois']))
        
    if i==2:
        break
        
test_annos = {"Mask R-CNN with ResNet101 backbone and Feature Pyramid Network (FPN)": test_annos}
#print(test_annos)
fd = open(JSON_PATH_OUTPUT, 'w')
json.dump(test_annos,fd,indent=4,sort_keys=True)

fd.close()